# TCSS 435 - Class Project: Can AI Think Like a Human?
Johan Hernandez
### Due: June 10th, 2025
---
## Step 1: Understanding LLMs & Transformers (20 pts)
- Explain how large language models are trained and how transformers
work
    - Large language models are trained using self-supervised learning on huge amounts of textual data. The first step in this process is generally to collect massive amounts of data from books, websites, wikipedia, and other textual sources. A large language model is then trained to predict the next token in a sequence of tokens, for example:
        ```txt
        Input: "I brushed my teeth with"
        Target: "Toothpaste"
        ```
        The LLM uses a transformer neural network where up to billions of parameters are adjusted through gradient descent and backpropagation to minimize error in predictions. Lastly, training requires high-performance GPUs or TPUs to train such a large model with high amounts of parameters and data, as training is resource intensive.
    - Transformers are the architecture behind most modern LLMs, as mentioned in the paper "Attention Is All You Need" by Ashish Vaswani et al. This architecture made handling context more efficiently and revolutionized natural language procesing. A transformer has input embedding, which converts words (tokens) into vectors using an embedding layer. They use positional encoding to know the order of the words, since they don't use recurrence like RNNs. The core mechanism of a transformer is multi-head attention, which allows the model to look at all other words in the input and weigh their relevance to the current word (see diagram and explanation below). After attention, the output is passed through a feedforward neural network to transform the representations. Transformers stack these layers many times to improve accuracy, for example GPT-3 uses 96+ layers 

- Include a diagram of a transformer model and explain 'attention'
    - The following is a diagram of the transformer model from "Attention Is All You Need" (Vaswani et al. 2017):

    ![Diagram of the transformer model](images/transformer-model.png)

    - Attention is what a LLM uses to know the relevance of tokens when compared with other tokens, or in other words what tokens to "pay attention" to. An attention function is described as "mapping a query and a set of key-value pairs to an output, where teh query, keys, and output are all vectors" (Vaswani et al. 2017). The following is a diagram escribing an attention funtion:

    ![Diagram of the attention model](images/attention-model.png)
    
    For each token, a model computes:
    $
    Attention(Q, K, V) = softmax(QK^T / \sqrt{d_k}) * V
    $
    Where Q is the query, K is the key, V is the value, and d_k is the dimension of key vectors. This formula weights how relevant each word is to the current one.

- Give 2 examples of real-world LLM applications
    - One real world example of an LLM is a coding assistant like GitHub Copilot. These models can generate code, explain logic, and autocomplete functions for developers. These LLMs specialize in not only natural language explanations, but also require a great deal of attention to code bases when dealing with coding suggestions and autocompletions.
    - Another real world LLM example is a customer support chatbot. These LLMs handle simple questions from customers about a company's products or services in natural language responses, unlike coding assistants.


## Step 2: Explore the CommonsenseQA Dataset (15 pts)
- Load dataset
- Print 5 random samples
- Identify commonsense types


In [1]:
from datasets import load_dataset
import random

# Load CommonsenseQA dataset from Hugging Face
dataset = load_dataset("commonsense_qa")
train_data = dataset['train']

# Display 5 random samples
samples = random.sample(list(train_data), 5)
for i, sample in enumerate(samples):
    print(f"Sample {i+1}:\nQuestion: {sample['question']}\nChoices: {sample['choices']['text']}\nAnswer: {sample['answerKey']}\n")


/home/johan/.pyenv/versions/ll/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Sample 1:
Question: If someone is clinging on to life they can be in a what?
Choices: ['ending soon', 'ill', 'death', 'coma', 'void']
Answer: D

Sample 2:
Question: Who is someone competing against?
Choices: ['effort', 'time', 'opponent', 'skill', 'competition']
Answer: C

Sample 3:
Question: The real estate alienated people, what did she need to learn to be?
Choices: ['deceive', 'charming', 'manipulate', 'lie', 'train']
Answer: B

Sample 4:
Question: Where do tourists frequent most in mexico?
Choices: ['beach', 'zoo', 'waterfall', 'homes', 'disneyland']
Answer: A

Sample 5:
Question: what does someone getting in shape want to achieve?
Choices: ['exercise', 'loss of muscle', 'losing weight', 'good health', 'sweat']
Answer: D



## Step 3: Build and Evaluate Baseline Models (15 pts)
- Random model
- Fixed-choice model
- Accuracy over 100 questions


In [2]:
# Implement random choice model
def random_choice_model(question, choices):
    return random.choice(choices)

# Evaluate the random choice model on 100 sample questions
correct_count = 0
for i in range(100):
    sample = random.choice(train_data)
    question = sample['question']
    choices = sample['choices']['label']
    correct_answer = sample['answerKey']
    
    # Get the model's prediction
    prediction = random_choice_model(question, choices)
    
    # Check if the prediction is correct
    if prediction == correct_answer:
        correct_count += 1
    if i < 5:  # Print first 5 predictions for verification
        print(f"Question: {question}\nChoices: {choices}\nPredicted Answer: {prediction}\nCorrect Answer: {correct_answer}\n")

# Calculate and print the accuracy
accuracy = correct_count / 100
print(f"Random Choice Model Accuracy: {accuracy:.2%}")


Question: The condominium has public recreational areas, what was the purpose of these?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: E
Correct Answer: E

Question: If I had more than one steel pen (e.g. 100,000), where would I store it?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: C
Correct Answer: B

Question: The machine was very intricate, it was quite an what?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: E
Correct Answer: B

Question: Killing will result in what kind of trial?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: D
Correct Answer: A

Question: What might someone shout when he or she is frustrated?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: C
Correct Answer: E

Random Choice Model Accuracy: 19.00%


In [3]:
# Implement a fixed choice model
def fixed_choice_model(question, choices):
    # Always return the first choice as the answer
    return choices[0]

# Evaluate the fixed choice model on 100 sample questions
correct_count_fixed = 0
for i in range(100):
    sample = random.choice(train_data)
    question = sample['question']
    choices = sample['choices']['label']
    correct_answer = sample['answerKey']
    
    # Get the model's prediction
    prediction = fixed_choice_model(question, choices)
    
    # Check if the prediction is correct
    if prediction == correct_answer:
        correct_count_fixed += 1
    if i < 5:  # Print first 5 predictions for verification
        print(f"Question: {question}\nChoices: {choices}\nPredicted Answer: {prediction}\nCorrect Answer: {correct_answer}\n")
# Calculate and print the accuracy for the fixed choice model
accuracy_fixed = correct_count_fixed / 100
print(f"Fixed Choice Model Accuracy: {accuracy_fixed:.2%}")

Question: Where might a milking machine be located?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: A
Correct Answer: C

Question: The dog had the sheep getting in line, it was amazing a dog could have what skills?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: A
Correct Answer: E

Question: The parents thought their children should learn teamwork, what were they signed up for?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: A
Correct Answer: B

Question: What is a person dressed as a clown hoping to accomplish?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: A
Correct Answer: B

Question: What would an adult woman do to get ready for work?
Choices: ['A', 'B', 'C', 'D', 'E']
Predicted Answer: A
Correct Answer: E

Fixed Choice Model Accuracy: 15.00%


### Interpretation
Both the random and fixed-choice models performed poorly on the subset of 100 random questions from the dataset. They both got anywhere between 15-25% of the answers correctly, which is expected. Both of the models have no sort of reasoning behind them, they both make a blind decision. Since there are 5 total choices for each of the questions and only one of the answers is correct, that gives any random choice theoretically a 20% chance of being choice. The same thing can be said for a fixed choice. If you always pick the same index as the answer, there is a theoretical 20% chance that you will guess correctly.

The reason why both models didn't score exactly 20% is due to the distribution of correct answers in the sampled questions. In practice, we would need to repeat this experiment thousands, millions, billions, or even more times (infinity) for the accuracy to begin approaching exactly 20%.

## Step 4: Prompting with LLMs (25 pts)
- Zero-shot and few-shot prompting
- Accuracy comparison

*For this part of the assignment, I chose to run a local instance of Llama 3.1 8B*

In [4]:
# Create prompts for the LLM to answer
prompts = []

for question in dataset['train']:
    choices = question['choices']['text']
    choice_str = "\n".join([f"{chr(ord('A') + i)}) {choice}" for i, choice in enumerate(choices)])
    prompt = (
        f"Answer the following questions using common sense reasoning.\n"
        f"Question: {question['question']}\n"
        f"Choices:\n{choice_str}\n"
        f"Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:"
    )
    prompts.append((prompt, question['answerKey']))

print(f"Total prompts created: {len(prompts)}")
print(prompts[:5])  # Display first 5 prompts for verification


Total prompts created: 9741
[('Answer the following questions using common sense reasoning.\nQuestion: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?\nChoices:\nA) ignore\nB) enforce\nC) authoritarian\nD) yell at\nE) avoid\nChoose the correct option (A, B, C, D, or E) respond ONLY with a single letter:', 'A'), ('Answer the following questions using common sense reasoning.\nQuestion: Sammy wanted to go to where the people were.  Where might he go?\nChoices:\nA) race track\nB) populated areas\nC) the desert\nD) apartment\nE) roadblock\nChoose the correct option (A, B, C, D, or E) respond ONLY with a single letter:', 'B'), ('Answer the following questions using common sense reasoning.\nQuestion: To locate a choker not located in a jewelry box or boutique where would you go?\nChoices:\nA) jewelry store\nB) neck\nC) jewlery box\nD) jewelry box\nE) boutique\nChoose the correct option (A, B, C, D, or E) respond ONLY w

In [16]:
# Implement zero shot prompting
from tqdm import tqdm
import ollama

# Define model parameters
MODEL = "llama3.1"
SYSTEM_PROMPT = "You are a helpful assistant that only response with a single letter: A, B, C, D, or E. Do not include explanations."
# Run inference using Ollama and LLaMA 3.1 8B model, zero-shot evaluation
results = []
for prompt, correct_answer in tqdm(prompts, desc="Evaluating with LLaMA 3.1 8B"):
    try:
        response = ollama.chat(
            model=MODEL,  # Make sure you've done `ollama pull llama3`
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': prompt}
            ]
        )
        answer = response['message']['content'].strip().upper()
        # Assume answer is something like "A", or "A. choice", extract the first letter
        answer_letter = answer[0] if answer and answer[0] in "ABCDE" else "?"
        is_correct = answer_letter == correct_answer
        results.append({
            "question": prompt,
            "predicted": answer_letter,
            "correct": correct_answer,
            "is_correct": is_correct
        })
    except Exception as e:
        print(f"Error processing prompt: {prompt}\nError: {e}")
        results.append({
            "question": prompt,
            "predicted": "ERROR",
            "correct": correct_answer,
            "is_correct": False,
            "error": str(e)
        })
        break
# Display th efirst 5 results
for result in results[:5]:
    print(f"Question: {result['question']}\nPredicted: {result['predicted']}\nCorrect: {result['correct']}\nIs Correct: {result['is_correct']}\n")

Evaluating with LLaMA 3.1 8B: 100%|██████████| 9741/9741 [15:35<00:00, 10.41it/s] 

Question: Answer the following questions using common sense reasoning.
Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
Choices:
A) ignore
B) enforce
C) authoritarian
D) yell at
E) avoid
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: A
Is Correct: True

Question: Answer the following questions using common sense reasoning.
Question: Sammy wanted to go to where the people were.  Where might he go?
Choices:
A) race track
B) populated areas
C) the desert
D) apartment
E) roadblock
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: B
Correct: B
Is Correct: True

Question: Answer the following questions using common sense reasoning.
Question: To locate a choker not located in a jewelry box or boutique where would you go?
Choices:
A) jewelry store
B) neck
C) jewlery box
D) jewelry box
E) boutique
Choose the correct o

*Screenshot just in case output disappears:*
![Zero shot output](images/zero-shot.png)

In [17]:
# Evaluate zero-shot accuracy
import pandas as pd

# Calculate accuracy
total = len(results)
correct = sum(r['is_correct'] for r in results)
accuracy = correct / total * 100

# Create results DataFrame
df_zero_shot = pd.DataFrame(results)
df_zero_shot["accuracy (%)"] = [accuracy] * len(df_zero_shot)  # Same value repeated for readability

# Show final accuracy
print(f"\nZero-Shot Accuracy with LLaMA 3.1 on dataset: {accuracy:.2f}%")



Zero-Shot Accuracy with LLaMA 3.1 on dataset: 65.73%


In [12]:
from tqdm import tqdm
import ollama

# Implement few-shot prompting
few_shot_questions = [question for question in random.sample(list(dataset['train']), 3)]
MODEL = "llama3.1"
SYSTEM_PROMPT = "You are a helpful assistant that only responds with a single letter: A, B, C, D, or E. Do not include explanations. The following are examples of questions and answers to help you understand the task:"
EXAMPLES = []
for question in few_shot_questions:
    choices = question['choices']['text']
    choice_str = "\n".join([f"{chr(ord('A') + i)}) {choice}" for i, choice in enumerate(choices)])
    example_prompt = (
        f"Question: {question['question']}\n"
        f"Choices:\n{choice_str}\n"
        f"Correct Answer: {question['answerKey']}\n"
    )
    EXAMPLES.append(example_prompt)

results = []

for prompt, correct_answer in tqdm(prompts, desc="Evaluating with Few-Shot LLaMA 3.1 8B"):
    try:
        full_system = SYSTEM_PROMPT + "\n\n" + "\n\n".join(EXAMPLES) + "\n\n" + prompt
        response = ollama.chat(
            model=MODEL,
            messages=[
                {'role': 'system', 'content': full_system},
                {'role': 'user', 'content': prompt}
            ]
        )
        answer = response['message']['content'].strip().upper()
        answer_letter = answer[0] if answer and answer[0] in "ABCDE" else "?"
        is_correct = answer_letter == correct_answer
        results.append({
            "question": prompt,
            "predicted": answer_letter,
            "correct": correct_answer,
            "is_correct": is_correct
        })
    except Exception as e:
        print(f"Error processing prompt: {prompt}\nError: {e}")
        results.append({
            "question": prompt,
            "predicted": "ERROR",
            "correct": correct_answer,
            "is_correct": False,
            "error": str(e)
        })
        break
# Display the first 5 results
for result in results[:5]:
    print(f"Question: {result['question']}\nPredicted: {result['predicted']}\nCorrect: {result['correct']}\nIs Correct: {result['is_correct']}\n")

Evaluating with Few-Shot LLaMA 3.1 8B: 100%|██████████| 9741/9741 [24:52<00:00,  6.52it/s] 

Question: Answer the following questions using common sense reasoning.
Question: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
Choices:
A) ignore
B) enforce
C) authoritarian
D) yell at
E) avoid
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: A
Is Correct: True

Question: Answer the following questions using common sense reasoning.
Question: Sammy wanted to go to where the people were.  Where might he go?
Choices:
A) race track
B) populated areas
C) the desert
D) apartment
E) roadblock
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: B
Correct: B
Is Correct: True

Question: Answer the following questions using common sense reasoning.
Question: To locate a choker not located in a jewelry box or boutique where would you go?
Choices:
A) jewelry store
B) neck
C) jewlery box
D) jewelry box
E) boutique
Choose the correct o

*Screenshot in case output from above cell disappears:*
![Few-shot output](images/few-shot.png)

In [14]:
import pandas as pd

# Evaluate few-shot accuracy
total = len(results)
correct = sum(r['is_correct'] for r in results)
accuracy = correct / total * 100
# Create results DataFrame
df_few_shot = pd.DataFrame(results)
df_few_shot["accuracy (%)"] = [accuracy] * len(df_few_shot)  # Same value repeated for readability
# Show final accuracy
print(f"\nFew-Shot Accuracy with LLaMA 3.1 on dataset: {accuracy:.2f}%")


Few-Shot Accuracy with LLaMA 3.1 on dataset: 72.19%


In [19]:
# Compare accuracies of zero-shot and few-shot prompting and report results
print("\nComparison of Zero-Shot and Few-Shot Prompting:")
print(f"Zero-Shot Accuracy: {df_zero_shot['accuracy (%)'].iloc[0]:.2f}%")
print(f"Few-Shot Accuracy: {df_few_shot['accuracy (%)'].iloc[0]:.2f}%")
print()
print("Results DataFrame for Zero-Shot Prompting:")
print(df_zero_shot.head())
print("\nResults DataFrame for Few-Shot Prompting:")
print(df_few_shot.head())
print()
print(f"Number of incorrect predictions in Zero-Shot: {len(df_zero_shot[df_zero_shot['is_correct'] == False])}")
print(f"Number of incorrect predictions in Few-Shot: {len(df_few_shot[df_few_shot['is_correct'] == False])}")


Comparison of Zero-Shot and Few-Shot Prompting:
Zero-Shot Accuracy: 65.73%
Few-Shot Accuracy: 72.19%

Results DataFrame for Zero-Shot Prompting:
                                            question predicted correct  \
0  Answer the following questions using common se...         A       A   
1  Answer the following questions using common se...         B       B   
2  Answer the following questions using common se...         A       A   
3  Answer the following questions using common se...         D       D   
4  Answer the following questions using common se...         C       C   

   is_correct  accuracy (%)  
0        True     65.732471  
1        True     65.732471  
2        True     65.732471  
3        True     65.732471  
4        True     65.732471  

Results DataFrame for Few-Shot Prompting:
                                            question predicted correct  \
0  Answer the following questions using common se...         A       A   
1  Answer the following questions usin

*Just in case comparing accuracy and results cell disappears, here's a screenshot of the output:*

![Comparison of zero-shot vs few-shot](images/comparison.png)

It may also be worth noting that the zero-shot prompts took roughly **15 minutes** in executing, while the few-shot prompts took roughly **25 minutes** in executing.

## Step 5: Error Analysis (15 pts)
- Choose 5 failed examples
- Reasoning or knowledge gap analysis


In [20]:
# Show 5 random incorrect predictions from both models
print("\nRandom Incorrect Predictions from Zero-Shot:")
incorrect_zero_shot = df_zero_shot[df_zero_shot['is_correct'] == False].sample(5)
for index, row in incorrect_zero_shot.iterrows():
    print(f"Question: {row['question']}\nPredicted: {row['predicted']}\nCorrect: {row['correct']}\n")
print("\nRandom Incorrect Predictions from Few-Shot:")
incorrect_few_shot = df_few_shot[df_few_shot['is_correct'] == False].sample(5)
for index, row in incorrect_few_shot.iterrows():
    print(f"Question: {row['question']}\nPredicted: {row['predicted']}\nCorrect: {row['correct']}\n")



Random Incorrect Predictions from Zero-Shot:
Question: Answer the following questions using common sense reasoning.
Question: Why might you be found grooming yourself in the mirror on the way out the door?
Choices:
A) cleanliness
B) mistakes
C) anxiety
D) beauty
E) neatness
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: E

Question: Answer the following questions using common sense reasoning.
Question: What is something bad unlikely to be to anyone?
Choices:
A) exceptional
B) virtuous
C) advantageous
D) strength
E) sufficient
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: C

Question: Answer the following questions using common sense reasoning.
Question: When someone isn't ridiculous at all they are what?
Choices:
A) straightforward
B) serious
C) sad
D) somber
E) solemn
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: B

Que

Here is the output from the above cell in case it disappears:

<small>
Random Incorrect Predictions from Zero-Shot:

Question: Answer the following questions using common sense reasoning.
Question: Why might you be found grooming yourself in the mirror on the way out the door?
Choices:
A) cleanliness
B) mistakes
C) anxiety
D) beauty
E) neatness
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: E

Question: Answer the following questions using common sense reasoning.
Question: What is something bad unlikely to be to anyone?
Choices:
A) exceptional
B) virtuous
C) advantageous
D) strength
E) sufficient
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: C

Question: Answer the following questions using common sense reasoning.
Question: When someone isn't ridiculous at all they are what?
Choices:
A) straightforward
B) serious
C) sad
D) somber
E) solemn
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: B

Question: Answer the following questions using common sense reasoning.
Question: Grapes are often grown in what sort of location in order to produce a beverage that impairs the senses when inbibed?
Choices:
A) rows
B) bowl of fruit
C) winery
D) painting
E) fruit stand
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: C

Question: Answer the following questions using common sense reasoning.
Question: John was cleaning out the old house.  while there was nothing upstairs, he found a bunch of old stuff somewhere else.  Where did he find stuff?
Choices:
A) loft
B) attic
C) museum
D) cellar
E) waste bin
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: B
Correct: D

--------------

Random Incorrect Predictions from Few-Shot:

Question: Answer the following questions using common sense reasoning.
Question: Why would someone be listening?
Choices:
A) empathy
B) thirsty
C) hear things
D) knowlege
E) learning
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: C

Question: Answer the following questions using common sense reasoning.
Question: If you catch your girlfriend lying about seeing another guy, you'll most likely experience what?
Choices:
A) broken heart
B) mistrust
C) getting dumped
D) being fired
E) get caught
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: C
Correct: A

Question: Answer the following questions using common sense reasoning.
Question: Residents socialized, played dice, and people smoked all at the busy landing of the what?
Choices:
A) airport
B) apartment building
C) stairwell
D) ocean
E) casino
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: A
Correct: B

Question: Answer the following questions using common sense reasoning.
Question: Where would one find a snake in a swamp?
Choices:
A) oregon
B) mud
C) tropical forest
D) pet store
E) louisiana
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: B
Correct: E

Question: Answer the following questions using common sense reasoning.
Question: Where might I place the lamp so it will help me see better while I do my homework?
Choices:
A) nearby
B) table
C) house
D) building
E) desktop
Choose the correct option (A, B, C, D, or E) respond ONLY with a single letter:
Predicted: B
Correct: E
</small>

### Zero-shot Analysis
Question 1: This question is asking why someone would groom themselves on the way out the door. The LLM answered A (cleanliness) when the correct answer was E (neatness). The LLM likely got this problem wrong due to how similar in meaning the two answers "cleaniness" and "neatness" are. On top of that, the LLM may be missing knowledge on human behavior such as grooming oneself.

Question 2: This question asks what something bad is unlikely to be to anyone, to which the LLM answered A (exceptional) when the correct answer was C (advantageous). The LLM likely also got this wrong due to how similar the answer was to its guess. On top of that, I believe the question required hidden knowledge that something bad may actually be exceptional for someone else.

Question 3: This question asks when someone is not ridiculous at all, they are what? The LLM answers A (straightforward) when the correct answer was B (serious). The words straightforward and serious are similar, with the difference that when someone is being straightforward they are simply telling the truth. However, when someone is being serious they are acting in a nature that means to not be joking, which the LLM may have overseen.

Question 4: This question asks the location of where grapes are grown when made to produce a beverage that impairs the senses (wine). The LLM answered A (rows) when the correct answer was C (winery). The LLM most likely got this question wrong due to missing the hint that the grapes are grown to make wine, and therefor are grown in a winery. It likely got it wrong to do the inability to associate a "drink that impairs your senses" with wine.

Question 5: This qusiton asks where John found a bunch of old stuff when cleaning out an old house. The LLM answered B (attic) when the correct answer was D (cellar). The reason the LLM most likely got this question wrong is due to the fact that the question states John found nothing upstairs, which means that there is a second story in the house and the attic is in the second story. Therefor, there was also nothing in the attic. The only other logical answer would be the cellar, the correct answer.

### Few-shot Analysis
Question 1: This question asks why would someone be listening. The LLM answers A (empathy) when the correct answer was C (hear things). The LLM likely got this one wrong likely due to it trying to connect emotions to the questions or the examples provided as hints included emotion in them, which led it to choose empathy instead of the straight up answer to hear things.

Question 2: This question asks what you would experience after catching your girlfriend lying about seeing another guy. The LLM answered C (getting dumped) when the correct answer was A (broken heart). THe LLM most likely got this one wrong due to its lack of training when dealing with human emotions and most likely associated "cheating" with a break up and chose "getting dumped". However, in this case you would experience some sort of heart break rather you yourself getting dumped, most liekly.

Question 3: This question asks where residents would socialize, smoke, and played dice at the landing of. The LLM answered A (airport) when the correct answer was B (apartment building). THe LLM most likely saw the word "landing" and immediately assumed the correct answer was "airport", thinking of a plane. But missed the word "residents" which implies they are somewhere communal, like an apartment building. More specifically the landing (an area between stairways) of an apartment building.

Question 4: This question asks where someone would find a snake in a swamp. The LLM answered B (mud) when the correct answer was E (louisiana). The LLM most likely does not have much context as to what exactly can be found in certain locations, such as states, and went with the next best answer "mud". It likely got this wrong due to a lack of specific information on geography and nature.

Question 5: This question asks where someone might place a lamp so it will help them see better while they do their homework. The LLM answered B (table) when the correct answer was E (desktop). The LLM most likely got this one wrong as a "table" is very broad and could include dinner tables and coffee tables. The LLM likely failed to recognize that homework is more likely to be done on a desktop and not just any table.

## Step 6: Final Report or Video (10 pts)
- Report key findings, surprising failures, reflections


### You can find Video here:

<iframe src="https://drive.google.com/file/d/1m2lVjRWFwTwo4zqWDUoqOgx2_IDeB9J6/preview" width="600" height="480"></iframe>

Or click <a href="https://drive.google.com/file/d/1m2lVjRWFwTwo4zqWDUoqOgx2_IDeB9J6/view">HERE</a>

Or here is a direct link:
https://drive.google.com/file/d/1m2lVjRWFwTwo4zqWDUoqOgx2_IDeB9J6/view

## Bonus: Chain-of-Thought Prompting (+5 pts)
- Try CoT on 20 examples
- Analyze performance impact
